In [ ]:
import sys
PATH = '/home/epuerta/Seroprevalence'
DIR_DATA = '/home/epuerta/Seroprevalence/'
sys.path.append(PATH) if PATH not in list(sys.path) else None
import os
import numpy as np
import pandas as pd
from scipy import interp
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns
from sklearn.metrics import plot_roc_curve
from imblearn.over_sampling import SMOTE
from sklearn import model_selection, preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import Lasso
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve,auc
from sklearn.metrics import r2_score

In [2]:
data = pd.read_csv(DIR_DATA + 'MC_CB.csv')

In [3]:
data = data.replace(np.nan, 0)
data.head()

,Edad,Sexo,raza,diagnosticado_covid,estado_civil,actividad_principal,contacto_sosp_covid,comparte_hab_per_cov,cuidador_persona_covid,familiar_fallecido_covid,...,dolor_garganta,fatiga_astenia,perdida_olfato,disminucion_gusto,comportamiento_pandemia,usa_tapabocas,tipo_tapabocas,cuantas_veces_limpia,veces_lava_manos,RESULTADO
0,48.0,2.0,1.0,0.0,4.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,1.0,1.0,4.0,3.0,0.0
1,54.0,1.0,1.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,1.0,1.0,2.0,3.0,0.0
2,62.0,2.0,1.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,1.0,1.0,2.0,3.0,0.0
3,22.0,2.0,1.0,0.0,1.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,1.0,1.0,2.0,3.0,0.0
4,46.0,1.0,1.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,1.0,2.0,3.0,3.0,0.0


In [4]:
data = data.drop(['raza', 'diagnosticado_covid'], axis = 1)

In [5]:
x = data.drop(['RESULTADO'], axis = 1)

In [6]:
col = list(x.columns)
len(col)

37

In [7]:
x = x.to_numpy()

In [8]:
min_max_scaler = MinMaxScaler()
x = min_max_scaler.fit_transform(x)

In [9]:
x

array([[0.6       , 1.        , 0.8       , ..., 0.33333333, 1.        ,
        1.        ],
       [0.675     , 0.5       , 0.2       , ..., 0.33333333, 0.5       ,
        1.        ],
       [0.775     , 1.        , 0.8       , ..., 0.33333333, 0.5       ,
        1.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [10]:
y = data['RESULTADO'].to_numpy()

In [11]:
y

array([0., 0., 0., ..., 0., 0., 0.])

In [12]:
print('**Train:', sorted(Counter(y).items()))

**Train: [(0.0, 1059), (1.0, 453)]


In [13]:
oversample = SMOTE()
x, y = oversample.fit_resample(x, y)

In [14]:
print('**OverSampling Train:', sorted(Counter(y).items()))

**OverSampling Train: [(0.0, 1059), (1.0, 1059)]


In [15]:
x

array([[0.6       , 1.        , 0.8       , ..., 0.33333333, 1.        ,
        1.        ],
       [0.675     , 0.5       , 0.2       , ..., 0.33333333, 0.5       ,
        1.        ],
       [0.775     , 1.        , 0.8       , ..., 0.33333333, 0.5       ,
        1.        ],
       ...,
       [0.77018955, 1.        , 0.56067396, ..., 0.33333333, 0.89957877,
        1.        ],
       [0.82959203, 0.5       , 0.8794066 , ..., 0.46567766, 1.        ,
        1.        ],
       [0.87469056, 1.        , 0.99504891, ..., 0.66666667, 0.75618887,
        0.67491849]])

In [16]:
x_feature = SelectKBest(chi2, k=4).fit_transform(x, y)

In [17]:
x_feature

array([[0.        , 0.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       ...,
       [1.        , 0.        , 0.        , 1.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.02475546]])

In [18]:
kfold = StratifiedShuffleSplit(n_splits=10, test_size=0.30, random_state=101)
# num_estimators = [20, 40, 60, 80, 100]
num_estimators = [20]

In [19]:
ensamble = {'Bagging': BaggingClassifier(n_estimators=20, random_state=42),
            'AdaBoost': AdaBoostClassifier(n_estimators=20, random_state=7),
            'GradientBoosting': GradientBoostingClassifier(n_estimators=20, random_state=7)}

# Without feature selection

In [20]:
print('-'*40)
for clf_name, clf in ensamble.items():
    auc = cross_val_score(clf, x, y, cv=kfold, scoring='roc_auc')
    acc_score = cross_val_score(clf, x, y, cv=kfold)
    recall_score = cross_val_score(clf, x, y, cv=kfold, scoring='recall')
    f1_score = cross_val_score(clf, x,y,cv=kfold, scoring='f1')
    print("Results for {} classifier".format(clf_name))
    print('-'*40)
    print("AUC: %0.2f (+/- %0.2f)" % ( auc.mean(), auc.std() * 2))
    print("Mean Accuracy: %0.3f (+/- %0.3f)" % (np.mean(acc_score), np.std(acc_score)) )
    print("Mean Recall: %0.3f (+/- %0.3f)" % (np.mean(recall_score), np.std(recall_score)) )
    print("Mean F1: %0.3f (+/- %0.3f)" % (np.mean(f1_score), np.std(f1_score)) )
    print('-'*40)

----------------------------------------
Results for Bagging classifier
----------------------------------------
AUC: 0.85 (+/- 0.02)
Mean Accuracy: 0.767 (+/- 0.009)
Mean Recall: 0.728 (+/- 0.017)
Mean F1: 0.758 (+/- 0.009)
----------------------------------------
Results for AdaBoost classifier
----------------------------------------
AUC: 0.81 (+/- 0.03)
Mean Accuracy: 0.713 (+/- 0.018)
Mean Recall: 0.770 (+/- 0.023)
Mean F1: 0.729 (+/- 0.012)
----------------------------------------
Results for GradientBoosting classifier
----------------------------------------
AUC: 0.82 (+/- 0.02)
Mean Accuracy: 0.726 (+/- 0.019)
Mean Recall: 0.732 (+/- 0.021)
Mean F1: 0.728 (+/- 0.015)
----------------------------------------


# Feature selection

In [21]:
print('-'*40)
for clf_name, clf in ensamble.items():
    auc = cross_val_score(clf, x_feature, y, cv=kfold, scoring='roc_auc')
    acc_score = cross_val_score(clf, x_feature, y, cv=kfold)
    recall_score = cross_val_score(clf, x_feature, y, cv=kfold, scoring='recall')
    f1_score = cross_val_score(clf, x_feature,y,cv=kfold, scoring='f1')
    print("Results for {} classifier".format(clf_name))
    print('-'*40)
    print("AUC: %0.2f (+/- %0.2f)" % ( auc.mean(), auc.std() * 2))
    print("Mean Accuracy: %0.3f (+/- %0.3f)" % (np.mean(acc_score), np.std(acc_score)) )
    print("Mean Recall: %0.3f (+/- %0.3f)" % (np.mean(recall_score), np.std(recall_score)) )
    print("Mean F1: %0.3f (+/- %0.3f)" % (np.mean(f1_score), np.std(f1_score)) )
    print('-'*40)

----------------------------------------
Results for Bagging classifier
----------------------------------------
AUC: 0.63 (+/- 0.03)
Mean Accuracy: 0.615 (+/- 0.017)
Mean Recall: 0.619 (+/- 0.019)
Mean F1: 0.617 (+/- 0.017)
----------------------------------------
Results for AdaBoost classifier
----------------------------------------
AUC: 0.63 (+/- 0.03)
Mean Accuracy: 0.616 (+/- 0.017)
Mean Recall: 0.621 (+/- 0.020)
Mean F1: 0.618 (+/- 0.017)
----------------------------------------
Results for GradientBoosting classifier
----------------------------------------
AUC: 0.64 (+/- 0.03)
Mean Accuracy: 0.616 (+/- 0.017)
Mean Recall: 0.621 (+/- 0.020)
Mean F1: 0.618 (+/- 0.017)
----------------------------------------
